# RNN - LSTM

## 1. Data Preprocessing

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
dataset_train = pd.read_csv('Google_Stock_Price_Train.csv')

In [3]:
training_set = dataset_train.iloc[:,1:2].values

In [4]:
training_set.shape

(1258, 1)

In [5]:
training_set[1:10]

array([[ 331.27],
       [ 329.83],
       [ 328.34],
       [ 322.04],
       [ 313.7 ],
       [ 310.59],
       [ 314.43],
       [ 311.96],
       [ 314.81]])

Feature scaling: Normalisation

In [7]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range=(0,1))

In [8]:
training_set_scaled = sc.fit_transform(training_set)

In [9]:
training_set_scaled[1:10]

array([[ 0.09701243],
       [ 0.09433366],
       [ 0.09156187],
       [ 0.07984225],
       [ 0.0643277 ],
       [ 0.0585423 ],
       [ 0.06568569],
       [ 0.06109085],
       [ 0.06639259]])

** Creating a data structure with 60 timesteps and 1 output **

In [10]:
X_train = []
y_train = []

In [13]:
for i in range(60, len(training_set_scaled)):
    X_train.append(training_set_scaled[i-60:i, 0])
    y_train.append(training_set_scaled[i, 0])

In [14]:
X_train, y_train = np.array(X_train), np.array(y_train)

In [16]:
X_train.shape, y_train.shape

((1198, 60), (1198,))

**Reshaping the data**

https://keras.io/layers/recurrent/

To add new indicators

In [18]:
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

In [19]:
X_train.shape

(1198, 60, 1)

## 2. Building the RNN

In [25]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

Initializing the RNN

In [28]:
regressor = Sequential()

Configuring the LSTM neural network

* return_sequences=True if after the LSTM layer goes another LSTM layer
* optimizer: recomended "RMSprop"; "adam" also ok 

In [29]:
regressor.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
regressor.add(Dropout(0.20))
regressor.add(LSTM(units=50, return_sequences=True))
regressor.add(Dropout(0.20))
regressor.add(LSTM(units=50, return_sequences=True))
regressor.add(Dropout(0.20))
regressor.add(LSTM(units=50))
regressor.add(Dropout(0.20))
regressor.add(Dense(units=1))
regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

In [31]:
# epochs = 100

regressor.fit(X_train, y_train, batch_size = 32, epochs = 10)

Epoch 1/10
1198/1198 [==============================] - 9s - loss: 0.0457     
Epoch 2/10
1198/1198 [==============================] - 8s - loss: 0.0064     
Epoch 3/10
1198/1198 [==============================] - 8s - loss: 0.0050     
Epoch 4/10
1198/1198 [==============================] - 8s - loss: 0.0053     
Epoch 5/10
1198/1198 [==============================] - 9s - loss: 0.0053     
Epoch 6/10
1198/1198 [==============================] - 9s - loss: 0.0053     
Epoch 7/10
1198/1198 [==============================] - 9s - loss: 0.0048     
Epoch 8/10
1198/1198 [==============================] - 9s - loss: 0.0046     
Epoch 9/10
1198/1198 [==============================] - 8s - loss: 0.0046     
Epoch 10/10
1198/1198 [==============================] - 9s - loss: 0.0045     


## 3. Making the predictions and visualising the results

Getting the real stock price of 2017

In [32]:
dataset_test = pd.read_csv('Google_Stock_Price_Test.csv')
real_stock_price = dataset_test.iloc[:,1:2].values

Getting the predicted stock price of 2017